In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep
import urllib.request


In [2]:
zonas = ['sul', 'norte', 'leste', 'oeste']
paginas = []
re_quarto ='\| (.*) quarto'
re_area = '> (.*) m²'


In [3]:
def pega_url(zona):
  zona = zona
  pag = 200
  paginas = []
  numero = 1
  lista_dfs = []
  url_req = ("https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-"+ zona +"/_Desde_"+ str(numero))
  while pag == 200:
    try:
      pag = urllib.request.urlopen(url_req).getcode()
      paginas.append(url_req)
      numero = numero + 48
      url_req = ("https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-"+ zona +"/_Desde_"+ str(numero))
      print(url_req)
    except:
      pag = 404
  return paginas


In [4]:
def coleta_dados(url):
    conteudo = requests.get(url).content
    sopa = BeautifulSoup(conteudo)
    
    precos = sopa.find_all('span', class_='price__fraction')
    areas_e_quartos = sopa.find_all('div', class_='item__attrs')
    
    lista_areas = []
    lista_quartos = []
    lista_precos = []
    lista_zonas = []
    
    for padrao in precos:
        preco = re.findall('<span class="price__fraction">(.*)</span>', str(padrao))
        lista_precos.append(preco[0].replace('.', ''))
    
    for elemento in areas_e_quartos:
        quarto = re.findall(re_quarto, str(elemento))
        area = re.findall(re_area, str(elemento))
        
        if quarto==[]:
            quarto=np.nan
        else:
            quarto=quarto[0]
        
        if area==[]:
            area=np.nan 
        else:
            area=area[0]

        lista_quartos.append(quarto)
        lista_areas.append(area)
        lista_zonas.append(zona)

    dados = {'zona': lista_zonas,
      'quartos': lista_quartos,
      'area': lista_areas,
      'preco': lista_precos}
  
    df = pd.DataFrame(dados)  
    return df


In [ ]:
lista_dfs = []
for zona in zonas:
  urls = pega_url(zona)
  for pagina in urls:
    #url = url_ml.format(zona, pagina)
    df = coleta_dados(pagina)
    print('Carregando a url', pagina)
    lista_dfs.append(df)
    sleep(2)
print('Dados coletados!')
dados = pd.concat(lista_dfs)
dados


https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145


In [ ]:
dados.to_csv('dados_mercadolivre_18082020.csv')
